In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
os.getcwd()

'/home/aryskin/ondewo/ondewo-t2s'

In [4]:
import io

In [5]:
import grpc
from ondewo_grpc.ondewo.t2s import text_to_speech_pb2, text_to_speech_pb2_grpc
import google.protobuf.empty_pb2 as empty_pb2
from google.protobuf.json_format import ParseDict, MessageToDict, MessageToJson

In [6]:
import soundfile as sf

In [7]:
import IPython.display as ipd

In [8]:
MAX_MESSAGE_LENGTH: int = 60000000
CAI_GRPC_HOST: str = "localhost"
CAI_GRPC_PORT: str = "50002"
CHANNEL: str = f"{CAI_GRPC_HOST}:{CAI_GRPC_PORT}"

options = [
    ('grpc.max_send_message_length', MAX_MESSAGE_LENGTH),
    ('grpc.max_receive_message_length', MAX_MESSAGE_LENGTH),
]


channel = grpc.insecure_channel(CHANNEL, options=options)
stab = text_to_speech_pb2_grpc.Text2SpeechStub(channel=channel)

In [9]:
id1, id2 = stab.ListActiveT2sPipelineIds(request=empty_pb2.Empty()).ids

In [10]:
request = text_to_speech_pb2.SynthesizeRequest(text='Hallo, wie geht es dir?', t2s_pipeline_id=id2, length_scale = 2.0, pcm=0)
response = stab.Synthesize(request=request)

bio = io.BytesIO(response.audio)

audio = sf.read(bio, )

ipd.Audio(audio[0], rate=audio[1])

In [11]:
request = text_to_speech_pb2.SynthesizeRequest(text='Hi, how are you?', t2s_pipeline_id=id1, pcm=0)
response = stab.Synthesize(request=request)

bio = io.BytesIO(response.audio)

audio = sf.read(bio)

ipd.Audio(audio[0], rate=audio[1])

In [12]:
request = text_to_speech_pb2.T2sPipelineId(id=id1)
response = stab.GetT2sPipeline(request=request)

In [13]:
config1 = MessageToDict(response, including_default_value_fields=True,preserving_proto_field_name=True)

In [14]:
config2 = MessageToJson(response, including_default_value_fields=True,preserving_proto_field_name=True)

In [15]:
config1['inference']['composite_inference']['text2mel']['glow_tts']['length_scale'] = 0.5

In [16]:
conf = ParseDict(config1,text_to_speech_pb2.Text2SpeechConfig())

In [17]:
stab.UpdateT2sPipeline(request=conf)

In [18]:
config1

{'id': 'glow_tts&hifi_gan-0118e07a-8447-4c87-9f6b-4802a7dd7a07',
 'description': 'German, male, thorsten dataset',
 'active': True,
 'inference': {'type': 'composite',
  'composite_inference': {'text2mel': {'type': 'glow_tts',
    'glow_tts': {'batch_size': '5',
     'use_gpu': True,
     'length_scale': 0.5,
     'noise_scale': 0.667,
     'path': 'models/glow-tts/en/pretrained.pth',
     'param_config_path': 'models/glow-tts/en/config.json',
     'cleaners': []},
    'glow_tts_triton': {'batch_size': '8',
     'length_scale': 1.0,
     'noise_scale': 0.667,
     'max_text_length': '100',
     'param_config_path': 'models/glow-tts/en/config.json',
     'triton_url': 'localhost:8001',
     'triton_model_name': 'glow_tts',
     'cleaners': []}},
   'mel2audio': {'type': 'hifi_gan',
    'waveglow_triton': {'param_config_path': 'models/waveglow/waveglow.yaml',
     'sigma': 0.6,
     'max_spect_size': '1000',
     'triton_model_name': 'waveglow',
     'triton_url': 'localhost:8001'},
    

In [19]:
type(response.inference.composite_inference.text2mel.glow_tts.batch_size)

int

In [20]:
type(conf.inference.composite_inference.text2mel.glow_tts.batch_size)

int